In [211]:
from random import choices
from typing import Dict, List, Tuple

import numpy as n
import pandas as pd

from Clasificador import Clasificador


class Individuo:
    def __init__(self, reglas: np.ndarray):
        self.reglas = reglas

    @staticmethod
    def crea_con_reglas_aleatorias(max_reglas: int, longitud_reglas: int):
        while True:
            reglas = np.random.randint(
                2, size=(np.random.randint(3, max_reglas), longitud_reglas)
            )
            # Verificar si alguna fila tiene solo 0s o solo 1s
            if not np.any(np.all(reglas == 0, axis=1)) and not np.any(
                np.all(reglas == 1, axis=1)
            ):
                break

        return Individuo(reglas)

    def clasifica_dato(self, dato: np.ndarray) -> int:
        votos = [0, 0]

        for regla in self.reglas:
            regla_activada = True

            for dato_bit, regla_bit in zip(dato[:-1], regla[:-1]):
                if dato_bit == 1 and regla_bit != 1:
                    regla_activada = False
                    break

            if regla_activada:
                votos[regla[-1]] += 1
        
        if sum(votos) == 0:
            return None
        elif votos[0] > votos[1]:
            return 0
        elif votos[0] < votos[1]:
            return 1
        else:
            return -1

    def fitness(self, datos_codificados: np.ndarray) -> float:
        aciertos = 0
        
        for dato in datos_codificados:
            prediccion = self.clasifica_dato(dato)

            if dato[-1] == prediccion:
                aciertos += 1

        return aciertos / datos_codificados.shape[0]

    # metodo prototipo
    def copia(self):
        return Individuo(np.copy(self.reglas))
        

# Revisar con el profesor si debemos de utilizar Datos
class CodificadorBinario:
    def __init__(self, datos: pd.DataFrame):
        self._n_bits, self._codificacion = self._init_codificacion(datos)

    def _init_codificacion(self, datos: pd.DataFrame) -> Tuple[int, Dict[str, Dict[str, List[int]]]]:
        columnas = datos.columns
        codificacion = {}
        n_bits = 0

        for columna in columnas[:-1]:
            valores = sorted(datos[columna].astype(str).unique())
            codificacion[columna] = {}

            for i, valor in enumerate(valores):
                codigo = [0] * len(valores)
                codigo[i] = 1

                codificacion[columna][valor] = codigo

            n_bits += len(valores)

        # Columna de clase, tiene una codificación con un solo bit
        codificacion[columnas[-1]] = {"0": [0], "1": [1]}
        n_bits += 1

        return n_bits, codificacion

    def codifica_datos(self, datos: pd.DataFrame) -> np.ndarray:
        filas_codificadas = []

        for _, fila in datos.iterrows():
            fila_codificada = []

            for columna, valor in fila.items():
                fila_codificada.extend(self._codificacion[columna][str(valor)])

            filas_codificadas.append(fila_codificada)

        return np.array(filas_codificadas)

    def n_bits(self) -> int:
        return self._n_bits


class AlgoritmoGenetico(Clasificador):
    def __init__(self, tamano_poblacion: int, epocas: int, max_reglas: int, porcentaje_elitismo: float):
        self.tamano_poblacion = tamano_poblacion
        self.epocas = epocas
        self.max_reglas = max_reglas
        self.n_elitistas = int(np.ceil(tamano_poblacion*porcentaje_elitismo))

    def entrenamiento(
        self, datosTrain: pd.DataFrame, nominalAtributos: List[bool], diccionario: Dict
    ):
        # Crea codificacion -> esto podria ser a traves de diccionario
        self.codificador = CodificadorBinario(datosTrain)
        
        # Crear primera generacion
        poblacion = self._crea_primera_generacion()
        datos_codificados = self.codificador.codifica_datos(datosTrain)

        for _ in range(self.epocas):
            # calcular fitness de la poblacion
            fitness_poblacion = self._evalua_fitness(datos_codificados, poblacion)

            # utilizar elitismo
            elite = self._selecciona_elite(poblacion, fitness_poblacion)

            # selecciona progenitores
            progenitores = self._selecciona_progenitores(poblacion, fitness_poblacion)
            cruzados = self._aplica_cruce_intra_reglas(progenitores)

            poblacion = elite + cruzados

        print(poblacion)
        print(self._evalua_fitness(datos_codificados, poblacion))
    
        # seleccionar padres
        # Crossovers => cruzar los padres
        # Mutaciones => mutar los padres para generar descendientes
        # Sobrevivientes => seleccionar solo los mejores descendientes
        # actualizar poblacion

        # calcular fitness de la poblacion final
        # calcular mejor solucion (mejor individuo)
    
    def _crea_primera_generacion(self) -> List[Individuo]:
        poblacion = []

        # Generar `tamano_poblacion` individuos con reglas aleatorias
        for _ in range(self.tamano_poblacion):
            poblacion.append(
                Individuo.crea_con_reglas_aleatorias(max_reglas=self.max_reglas, longitud_reglas=self.codificador.n_bits())
            )

        return poblacion

    def _evalua_fitness(self, datos_codificados: np.ndarray, poblacion: List[Individuo]) -> float:
        fitness_poblacion = [individuo.fitness(datos_codificados) for individuo in poblacion]
        return fitness_poblacion

    def _selecciona_elite(self, poblacion: List[Individuo], fitness_poblacion: List[float]) -> List[Individuo]:
        # Ordenar la población según la aptitud (mayor aptitud primero)
        poblacion_ordenada = [individuo for _, individuo in sorted(zip(fitness_poblacion, poblacion), key=lambda x: x[0], reverse=True)]

        # Seleccionar a los mejores individuos (élite)
        elite = poblacion_ordenada[:self.n_elitistas]

        return elite
    
    # Esta parte necesesita la codificacion terminada
    def _selecciona_progenitores(
        self, poblacion: List[Individuo], fitness_poblacion: List[float]
    ) -> List[Individuo]:
        n_progenitores = self.tamano_poblacion - self.n_elitistas

        # Normalizar la aptitud para convertirla en probabilidades
        probabilidad_seleccion = [fitness / sum(fitness_poblacion) for fitness in fitness_poblacion]

        # Utilizar np.random.choice para seleccionar progenitores
        progenitores_indices = np.random.choice(np.arange(len(poblacion)), size=n_progenitores, p=probabilidad_seleccion)
        progenitores = [poblacion[i].copia() for i in progenitores_indices]

        return progenitores

    def _aplica_cruce_intra_reglas(self, poblacion: List[Individuo]) -> List[Individuo]:
        cruzados = []

        for _ in range(len(poblacion) // 2):
            # Seleccionar dos progenitores aleatorios
            progenitor1, progenitor2 = np.random.choice(poblacion, size=2, replace=False)

            # Realizar el cruce intra reglas
            punto_cruce = np.random.randint(min(len(progenitor1.reglas), len(progenitor2.reglas)))

            nueva_reglas1 = np.vstack((progenitor1.reglas[:punto_cruce], progenitor2.reglas[punto_cruce:]))
            nueva_reglas2 = np.vstack((progenitor2.reglas[:punto_cruce], progenitor1.reglas[punto_cruce:]))

            hijo1 = Individuo(nueva_reglas1)
            hijo2 = Individuo(nueva_reglas2)

            cruzados.extend([hijo1, hijo2])

        return cruzados

In [212]:
import pandas as pd

df_titanic = pd.read_csv("titanic.csv")

In [213]:
clasificador = AlgoritmoGenetico(
    tamano_poblacion=10,
    epocas=10,
    max_reglas=5,
    porcentaje_elitismo=0.05
)

In [214]:
clasificador.entrenamiento(datosTrain=df_titanic, nominalAtributos=[], diccionario={})

[<__main__.Individuo object at 0x7fc81f1b2290>, <__main__.Individuo object at 0x7fc81fbba5d0>, <__main__.Individuo object at 0x7fc823b83750>, <__main__.Individuo object at 0x7fc81f094b90>, <__main__.Individuo object at 0x7fc81fdee5d0>, <__main__.Individuo object at 0x7fc81ff107d0>, <__main__.Individuo object at 0x7fc81f062ed0>, <__main__.Individuo object at 0x7fc821153d10>, <__main__.Individuo object at 0x7fc81f1b30d0>]
[0.381453154875717, 0.36615678776290633, 0.381453154875717, 0.381453154875717, 0.36615678776290633, 0.381453154875717, 0.36615678776290633, 0.36615678776290633, 0.381453154875717]


In [116]:
cb = CodificadorBinario(df_titanic)

In [117]:
a = cb.codifica_datos(df_titanic)

In [131]:
ind = Individuo.crea_con_reglas_aleatorias(max_reglas=10, longitud_reglas=cb.n_bits())

In [132]:
a

array([[0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [133]:
ind.fitness(a)

0.25908221797323133

In [135]:
a[0]

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])

In [136]:
ind.reglas

array([[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]])